In [2]:
import pymysql
import pandas as pd
import numpy as np
import matplotlib as mp
from sqlalchemy import create_engine
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey, func
from sqlalchemy.orm import sessionmaker, relationship, Session
from sqlalchemy.ext.automap import automap_base
import os

## Load Files

In [3]:
pwd

'C:\\Users\\Ryan\\OneDrive - ogg.com\\DU_class\\Caffeine-Project\\Caffeine'

In [4]:
csv1 = "CSVs/All_drink_info.csv"
csv2 = "CSVs/Caffeine_intensity.csv"
csv3 = "CSVs/Workout Supps.csv"
csv4 = "CSVs/Food.csv"
csv5 = "CSVs/Gum.csv"

## Preview Files

In [5]:
alldrinks = pd.read_csv(csv1)
caff_int = pd.read_csv(csv2)
supps = pd.read_csv(csv3)
food = pd.read_csv(csv4)
gum = pd.read_csv(csv5)

In [6]:
alldrinks.head()

,drinks,caff_cont,fluid_oz,mg_per_oz,url
0,10 Hour Energy Shot,422,1.93,218.7,https://www.caffeineinformer.com/caffeine-cont...
1,28 Black Energy Drink,80,8.46,9.5,https://www.caffeineinformer.com/caffeine-cont...
2,3 Water,50,16.90,3.0,https://www.caffeineinformer.com/caffeine-cont...
3,3D Energy Drink,200,16.00,12.5,https://www.caffeineinformer.com/caffeine-cont...
4,4 Purpose Energy Drink,70,8.46,8.3,https://www.caffeineinformer.com/caffeine-cont...


In [7]:
caff_int.head()

,caff_str,item_img,drink_name
0,DANGEROUS,https://caffeineinformer.com/wp-content/caffei...,10 Hour Energy Shot
1,MODERATE,https://caffeineinformer.com/wp-content/caffei...,28 Black Energy Drink
2,MODERATE,https://caffeineinformer.com/wp-content/caffei...,3 Water
3,VERY HIGH,https://caffeineinformer.com/wp-content/caffei...,3D Energy Drink
4,MODERATE,https://caffeineinformer.com/wp-content/caffei...,4-Purpose


In [8]:
supps.head()

,supp,caff_source,serving_size,caff
0,ABB Speed Stacked Pumped,"Caffeine Anhydrous, Guarana",bottle,300
1,About Time AUX,"Yerba Mate’, Guarana, Green Tea",scoop,60
2,BANG Energy Drink,Caffeine Anhydrous,can,300
3,Hyde Power Potion,Caffeine Anhydrous,can,350
4,BPI B4,Caffeine Anhydrous,capsule,300


In [9]:
supps.dtypes

supp            object
caff_source     object
serving_size    object
caff             int64
dtype: object

In [10]:
food.head()

,food_name,caff_per_mg,serving_size,url,img_url
0,AeroLife Inhalable Caffeine,100,per dose,https://www.caffeineinformer.com//caffeine-con...,https://cdn.caffeineinformer.com/wp-content/ca...
1,Alert Caffeine Gum,40,per piece,https://www.caffeineinformer.com//caffeine-con...,https://cdn.caffeineinformer.com/wp-content/ca...
2,Alien Energy Jerky,110,per packs,https://www.caffeineinformer.com//caffeine-con...,https://cdn.caffeineinformer.com/wp-content/ca...
3,Anacin,32,per tablet,https://www.caffeineinformer.com//caffeine-con...,https://cdn.caffeineinformer.com/wp-content/ca...
4,Apples,0,per apple,https://www.caffeineinformer.com//caffeine-con...,https://cdn.caffeineinformer.com/wp-content/ca...


In [11]:
gum.head()

,Gum_name,Caff_per_piece,Price_per_pack,Page_url,Flavor
0,Military Energy Gum\n(Formerly Stay Alert Gum),100.0,1.28,https://www.caffeineinformer.com/caffeine-cont...,mint
1,Military Energy Gum\n(Formerly Stay Alert Gum),100.0,1.28,https://www.caffeineinformer.com/caffeine-cont...,cinnamon
2,Apollo Energy Gum,80.0,2.60,http://www.apolloenergygum.com/,spearmint
3,Blitz Energy Gum,55.0,2.45,None,mint original
4,X8 Energy Gum,50.0,4.99,https://www.caffeineinformer.com/caffeine-cont...,mint


In [13]:
# Create the Engine
engine = create_engine("sqlite:///caffeine.sqlite")

In [14]:
# Create Base
Base = declarative_base()

In [15]:
# Create ORMS for table creation

class Drinks(Base):
    
    __tablename__ = 'drinks'
    
    id = Column(Integer, primary_key=True)
    drinks = Column(Text)
    caff_cont = Column(Integer)
    fluid_oz = Column(Float)
    mg_per_oz = Column(Float)
    url = Column(Text)
    
    def __repr__(self):
        return f"id={self.id}, name={self.drinks}"
    
class Caffeine_info(Base):
    
    __tablename__ = 'caff_info'
    
    id = Column(Integer, primary_key=True)
    caff_str = Column(Text)
    item_img = Column(Text)
    drink_name = Column(Text)
    
    def __repr__(self):
        return f"id={self.id}, name={self.caff_str}"

class Supplements(Base):
    
    __tablename__ = 'supplements'
    
    id = Column(Integer, primary_key=True)
    supp = Column(Text)
    caff = Column(Integer)
    caff_source = Column(Text)
    serving_size = Column(Text)
    
    def __repr__(self):
        return f"id={self.id}, name={self.supp}"
    

class Food(Base):
    
    __tablename__ = 'food'
    
    id = Column(Integer, primary_key=True)
    food_name = Column(Text)
    caff_per_mg = Column(Float)
    serving_size = Column(Text)
    url = Column(Text)
    img_url = Column(Text)
    
    def __repr__(self):
        return f"id={self.id}, name={self.food_name}"

class Gum(Base):
    
    __tablename__ = 'gum_info'
    
    id = Column(Integer, primary_key=True)
    Gum_name = Column(Text)
    Caff_per_piece = Column(Float)
    Flavor = Column(Text)
    Price_per_pack = Column(Float)
    Page_url = Column(Text)
    
    def __repr__(self):
        return f"id={self.id}, name={self.Gum_name}"
    

In [16]:
# Create tables
Base.metadata.create_all(engine)

In [17]:
# List table names to check if they are created
engine.table_names()

['caff_info', 'drinks', 'food', 'gum_info', 'supplements']

In [18]:
# Function to add data
def populate_table(engine,table,csvfile):
    
    conn = engine.connect()
    df_to_insert = pd.read_csv(csvfile)
    data = df_to_insert.to_dict(orient='records')
    conn.execute(table.delete())
    conn.execute(table.insert(),data)

In [19]:
# Testing function on one table
populate_table(engine,Drinks.__table__,"CSVs/All_drink_info.csv")

In [20]:
# Passing raw SQL query to test if the load was successful
engine.execute("SELECT * FROM drinks LIMIT 10").fetchall()

[(1, '10 Hour Energy Shot', 422, 1.93, 218.7, 'https://www.caffeineinformer.com/caffeine-content/10-hour-eternal-shot'),
 (2, '28 Black Energy Drink', 80, 8.46, 9.5, 'https://www.caffeineinformer.com/caffeine-content/28-energy-drink-black-white'),
 (3, '3 Water ', 50, 16.9, 3.0, 'https://www.caffeineinformer.com/caffeine-content/3-caffeine-water'),
 (4, '3D Energy Drink', 200, 16.0, 12.5, 'https://www.caffeineinformer.com/caffeine-content/3d-energy-drink'),
 (5, '4 Purpose Energy Drink', 70, 8.46, 8.3, 'https://www.caffeineinformer.com/caffeine-content/4-purpose'),
 (6, '4C Energy Liquid Water Enhancers', 60, 8.0, 7.5, 'https://www.caffeineinformer.com/caffeine-content/4c-energy-water-enhancers'),
 (7, '4C Energy Rush', 80, 8.0, 10.0, 'https://www.caffeineinformer.com/caffeine-content/4-c-energy-rush'),
 (8, '5 Hour Energy', 200, 2.0, 100.0, 'https://www.caffeineinformer.com/caffeine-content/5-hour-energy'),
 (9, '5 Hour Energy Decaf', 6, 2.0, 3.0, 'https://www.caffeineinformer.com/caf

In [21]:
# Populating the tables
populate_table(engine,Caffeine_info.__table__,"CSVs/Caffeine_intensity.csv")
populate_table(engine,Supplements.__table__,"CSVs/Workout Supps.csv")
populate_table(engine,Food.__table__,"CSVs/Food.csv")
populate_table(engine,Gum.__table__,"CSVs/Gum.csv")

In [22]:
# Preparing the tables for query testing
Base = automap_base()
Base.prepare(engine, reflect=True)

In [23]:
Base.classes.keys()

['caff_info', 'drinks', 'food', 'gum_info', 'supplements']

In [24]:
Drinks = Base.classes.drinks
session = Session(engine)

In [25]:
drink_list = session.query(Drinks).\
    filter(Drinks.caff_cont >= 400).all()

In [26]:
drinklist = []
for d in drink_list:
    minitem = {}
    minitem['Drink Name'] = d.drinks
    minitem['Caffeine Content'] = d.caff_cont
    minitem['Serving Size Fluid OZ'] = d.fluid_oz
    minitem['MG per oz'] = d.mg_per_oz
    drinklist.append(minitem)
drinklist

[{'Drink Name': '10 Hour Energy Shot',
  'Caffeine Content': 422,
  'Serving Size Fluid OZ': 1.93,
  'MG per oz': 218.7},
 {'Drink Name': 'Biohazard Coffee',
  'Caffeine Content': 928,
  'Serving Size Fluid OZ': 12.0,
  'MG per oz': 77.3},
 {'Drink Name': 'Bizzy Cold Brew',
  'Caffeine Content': 750,
  'Serving Size Fluid OZ': 16.0,
  'MG per oz': 46.9},
 {'Drink Name': 'Black Insomnia Coffee',
  'Caffeine Content': 1105,
  'Serving Size Fluid OZ': 12.0,
  'MG per oz': 92.1},
 {'Drink Name': 'Black Label Brewed Coffee',
  'Caffeine Content': 1555,
  'Serving Size Fluid OZ': 12.0,
  'MG per oz': 129.6},
 {'Drink Name': 'Cannonball Coffee Maximum Charge',
  'Caffeine Content': 1101,
  'Serving Size Fluid OZ': 12.0,
  'MG per oz': 91.8},
 {'Drink Name': 'Coffee Crave Fearless Black',
  'Caffeine Content': 844,
  'Serving Size Fluid OZ': 12.0,
  'MG per oz': 70.3},
 {'Drink Name': 'Death Wish Coffee',
  'Caffeine Content': 728,
  'Serving Size Fluid OZ': 12.0,
  'MG per oz': 60.7},
 {'Drin